In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,sklearn,tf,keras:
    print(module.__name__,module.__version__)
	

1.15.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
sklearn 0.22.2.post1
tensorflow 1.15.0
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test)=fashion_mnist.load_data()
x_valid,x_train=x_train_all[:5000],x_train_all[5000:]
y_valid,y_train=y_train_all[:5000],y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

print(np.max(x_train),np.min(x_train))

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)
255 0


In [3]:
# 归一化
from sklearn.preprocessing import StandardScaler

scaler =StandardScaler()
# x_train: [None,28,28]-> [None,784]
x_train_scaled=scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_valid_scaled=scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_test_scaled=scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

y_train=np.asarray(y_train,dtype=np.int64)
y_valid=np.asarray(y_valid,dtype=np.int64)
y_test=np.asarray(y_test,dtype=np.int64)

In [4]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [5]:
def make_dataset(images, labels, epochs ,bs , shuffle =True):
    dataset= tf.data.Dataset.from_tensor_slices((images,labels))
    if shuffle:
        dataset=dataset.shuffle(10000)
    dataset=dataset.repeat(epochs).batch(bs)
    return dataset    


In [6]:
bs =20
epochs=10
dataset=make_dataset(x_train_scaled,y_train,epochs=epochs,bs=bs)
for data,label in dataset.take(1):
    print(data)
    print(label)

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [7]:
hidden_units=[100,100]
class_num=10
input_for_next_layer=x
for hidden_unit in hidden_units:
    input_for_next_layer=tf.layers.dense(input_for_next_layer,hidden_unit,activation=tf.nn.relu)

logits=tf.layers.dense(input_for_next_layer,class_num)

#  last_hidden_output*W(logits)-> softmax->prob
#  1. logit -> softmax ->prob
# 2. labels -> one_hot
# 3. calculate cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)

# get accuracy
prediction = tf.argmax(logits,1)
crrect_prediction=tf.equal(prediction,y)
accuracy= tf.reduce_mean(tf.cast(crrect_prediction,tf.float64))

train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)

NameError: name 'x' is not defined

In [ ]:
bs =20
epochs=10
dataset=make_dataset(x_train_scaled,y_train,epochs=epochs,bs=bs)
dataset_iter=dataset.make_one_shot_iterator()
x,y=dataset_iter.get_next()

with tf.Session() as sess:
    x_val,y_val=sess.run([x,y])
    print(x_val.shape)
    print(y_val.shape)

In [ ]:
print(x)
print(logits)


In [ ]:
# session
init = tf.global_variables_initializer()

train_steps_per_epoch=x_train.shape[0]//bs

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            loss_val, accuracy_val, _=sess.run([loss,accuracy,train_op])
            print('\r[Train] epoch: %d,step: %d, loss: %3.5f,accuracy: %2.2f'%(epoch,step,loss_val,accuracy_val),end="")